In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, models
from torchvision.models import ResNet101_Weights

from utils.disc21 import DISC21Definition, DISC21

In [2]:
model = models.resnet101(weights=ResNet101_Weights.DEFAULT)
model.fc = torch.nn.Identity()
model.avgpool = torch.nn.Identity()

In [3]:
transformation_chain = transforms.Compose(
    [
        # We first resize the input image to 256x256, and then we take center crop.
        transforms.Resize(int((256 / 224) * 224)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

In [4]:
train_df = DISC21Definition('/media/augustinas/T7/DISC2021/SmallData/images/')
train_ds = DISC21(train_df, subset='gallery', transform=transformation_chain, augmentations=None)

DISC21Definition dataset loaded
  subset   | # ids | # images
  ---------------------------
  train    | 100000 |   100000
  gallery  | 100000 |   100000
  query    |  10000 |    10000


In [5]:
train_loader = DataLoader(train_ds, batch_size=1, shuffle=False, num_workers=1)

In [19]:
for i, (anchor, y) in enumerate(train_loader):
    print(i)
    print(anchor.shape)
    print(y[0])
    with torch.no_grad():
        anchor_out = model(anchor)
    print(anchor_out)
    bottleneck_values = np.squeeze(anchor_out)
    print(bottleneck_values.shape)
    bottleneck_values = bottleneck_values / np.linalg.norm(bottleneck_values)
    print(bottleneck_values.shape)
    np.save(f'./resnet_data/r/{y[0][:-4]}.npy', bottleneck_values)
    break

0
torch.Size([1, 3, 224, 224])
R48127.jpg
tensor([[0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([100352])
torch.Size([100352])
